In [14]:
import tkinter as tk #tk for file dialog (requires Jinja2!!!)
from tkinter import filedialog #tkinter for file dialog

import re #regex for parsing
import numpy as np
import pandas as pd
import os
from os.path import exists
import torch

def isTextFile(filepath):
	fLen = len(filepath)
	#Check that our filepath is a .txt
	return bool((fLen > 4) and (filepath[fLen-4:] == ".txt"))

In [15]:
#any data that doesn't make a full epoch gets cut off im pretty sure (because the epoch tensor is size INT(maxSize/epoch_size))
def getTensorFromText(filepath, maxSize=None, epoch_size = 100):## just change the 
	#Check that our filepath is a .txt")
    if (isTextFile(filepath)):
        dataFp = open(filepath, "r")
        readingData = False
        col1, col2, col3,  col4 = [], [], [], []

        line_count = 0

        epoch_tensor = torch.zeros(int(maxSize/epoch_size), 3, epoch_size)#tensor to hold each epoch of data
        epoch_start = 0
        epoch_end = epoch_size
        epoch_index = 0

        for line in dataFp.readlines():
            if ((maxSize != None) and (maxSize <= line_count)):
                break

            if epoch_start >= epoch_end:
                epoch = torch.Tensor([col1, col2, col3])#im just getting one of the time series columns for now for simplicities sake
                epoch_tensor[epoch_index] = epoch
                col1, col2, col3, col4 = [], [], [], []                
                epoch_end += epoch_size
                epoch_index += 1

            if readingData:                                        
				#split the line by tabs and add the data to our column lists
                data_list = line.split("\t")
                stripped_time = re.sub("\s", "",data_list[1])
                time = stripped_time.split(":")

                col1.append(int(time[0]))#idk how much it matters for storage but it might be easier to multiply this by 60
                col2.append(np.float32(time[1]))#i think we do need this to be precise because the timesteps should be consitent so i changed it to 32
                col3.append(np.float16(re.sub("\s", "",data_list[2])))
                col4.append(np.float16(re.sub("\s", "",data_list[3])))

                line_count += 1
                epoch_start += 1
                
				#Ignore initial lines
            else:		
                stripped = re.sub("\s", "", line)
					#This line is the last line before data comes
                if stripped == "(m:s.ms)(mV)(mV)":
                    readingData = True

        dataFp.close()

        return epoch_tensor
    else:
        print("Cannot handle file", filepath)
        return None	

In [16]:
eeg_tensor = getTensorFromText("../CHDCtrl1_CHD801FR_normal/CHD801FR_20221123_normal.txt", maxSize =1000000, epoch_size = 200)

In [17]:
eeg_df = pd.DataFrame(eeg_tensor[:, :, 0]).astype("float")
end_df = pd.DataFrame(eeg_tensor[:, :, 199]).astype("float")

eeg_df = eeg_df.rename(columns={0: "min", 1: "sec", 2:"sig"})
eeg_df["min"] = (eeg_df["min"]*60) + eeg_df.sec

end_df = end_df.rename(columns={0: "min", 1: "sec", 2:"sig"})
end_df["sec"] = (end_df["min"]*60) + end_df.sec

eeg_df["sec"] = end_df["sec"]
eeg_df = eeg_df.rename(columns={"min": "start", "sec": "end"})

In [18]:
def getStatesOverTime(filepath):## just change the 
	#Check that our filepath is a .txt")
    time, state = [], []
    
    data_an = open(filepath)#open and read file
    r = data_an.read()
    data_an.close()

    rows = list(r.split("\n"))#list of each row (strings)
    for row in rows:
        row_list = list(row.split(","))
        if row_list == [""]:
            break

        time.append(np.float32(row_list[2]))
        state.append(int(row_list[5]))

    return [time, state]

In [19]:
statesOverTime = getStatesOverTime("../CHDCtrl1_CHD801FR_normal/CHD801FR_20221123_normal_annotated.txt")

In [20]:
def add_state_column(s = 0, e = 10000):

    states_col = []
    epoch_i = 1
    time_state_i = 1

    for index, row in eeg_df[s:e].iterrows():
        epoch_start = row["start"]
        epoch_end = row["end"]

        if epoch_end >= statesOverTime[0][time_state_i]:
            
            if epoch_start < statesOverTime[0][time_state_i]:
                states_col.append(4)

            else:
                states_col.append(statesOverTime[1][time_state_i])
            time_state_i += 1

        else:
            states_col.append(statesOverTime[1][time_state_i])

        epoch_i += 1
  
    return(states_col)

In [21]:
labels = add_state_column()

In [22]:
ds = torch.utils.data.TensorDataset(eeg_tensor, torch.tensor(labels))
train_loader = torch.utils.data.DataLoader(ds, batch_size=64, shuffle=True)

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [24]:
import torch.nn.functional as F
from torch import nn, optim

class Model_3(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(Model_3, self).__init__()
        self.n_feature = n_feature
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=n_feature, kernel_size=(3))
        self.fc1 = nn.Linear(135168, 10)#64*28*28
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x, verbose=False):
        #print(x.size())
        x = self.conv1(x)
        x = F.relu(x)
        #print(x.size())
        x = F.max_pool2d(x, kernel_size=(1, 3))
        #print(x.size())
        x = F.relu(x)
        #x = F.max_pool2d(x, kernel_size=(1, 3))
        #print(x.size())
        x = x.size()[0]
        x = x.view(x, -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [25]:
def train_model(epochs, model):
    model.train() #set model to training mode
    loss_fun = nn.CrossEntropyLoss() #define a loss function object

    for epoch in range(epochs):

      for batch_idx, (data, target) in enumerate(train_loader):
          # send to device
          data, target = data.to(device), target.to(device)

          
          optimizer.zero_grad()
          output = model(data)
          loss = loss_fun(output,target)
          loss.backward()
          optimizer.step()
          if batch_idx % 100 == 0:
              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(trainloader.dataset),
                  100. * batch_idx / len(trainloader), loss.item()))

In [26]:
model_step_3 = Model_3(3*200, 32, 4)
model_step_3.to(device)
  
optimizer = torch.optim.SGD(model_step_3.parameters(), lr=.01)
train_model(10, model_step_3)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)